# LH 공고 RAG 챗봇

PostgreSQL + pgvector를 사용한 임대/분양 공고 질의응답 시스템

## 사전 준비사항

### 1. Docker로 DB 실행
```bash
cd deployment
docker-compose up -d
```

### 2. 필요한 패키지 설치
```bash
pip install psycopg2-binary asyncpg sentence-transformers openai python-dotenv pandas
```

### 3. OpenAI API 키 설정 (LLM 질의응답 사용 시)
프로젝트 루트에 `.env` 파일 생성 후 다음 내용 추가:
```
OPENAI_API_KEY=sk-your-api-key-here
```

In [1]:
!pip install psycopg2-binary asyncpg sentence-transformers openai python-dotenv pandas

## 1. 라이브러리 임포트 및 설정

In [2]:
import asyncio
import asyncpg
from sentence_transformers import SentenceTransformer
import pandas as pd
import json
from IPython.display import display, Markdown

# DB 설정 (Docker 기본값)
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'database': 'skn19_3rd_proj',
    'user': 'rag_user',
    'password': 'skn19'
}

# 임베딩 모델 설정
MODEL_NAME = 'BAAI/bge-m3'

print("임베딩 모델 로딩 중...")
model = SentenceTransformer(MODEL_NAME)
print("모델 로딩 완료")

임베딩 모델 로딩 중...
모델 로딩 완료


## 2. 데이터베이스 연결 확인

In [3]:
async def show_db_stats():
    """데이터베이스 통계 확인"""
    try:
        conn = await asyncpg.connect(**DB_CONFIG)
        
        # 전체 통계
        total_announcements = await conn.fetchval(
            "SELECT COUNT(*) FROM announcements"
        )
        total_chunks = await conn.fetchval(
            "SELECT COUNT(*) FROM document_chunks"
        )
        vectorized_count = await conn.fetchval(
            "SELECT COUNT(*) FROM announcements WHERE is_vectorized = TRUE"
        )
        
        print("=" * 60)
        print("데이터베이스 통계")
        print("=" * 60)
        print(f"총 공고 수: {total_announcements:,}개")
        print(f"벡터화 완료: {vectorized_count:,}개 ({vectorized_count/total_announcements*100:.1f}%)")
        print(f"총 청크 수: {total_chunks:,}개")
        if vectorized_count > 0:
            print(f"평균 청크/공고: {total_chunks/vectorized_count:.1f}개")
        
        # 카테고리별 통계
        category_stats = await conn.fetch("""
            SELECT 
                category,
                COUNT(*) as total,
                SUM(CASE WHEN is_vectorized THEN 1 ELSE 0 END) as vectorized
            FROM announcements
            GROUP BY category
        """)
        
        print("\n분류별 벡터화 현황:")
        for row in category_stats:
            cat_name = "임대" if row['category'] == 'lease' else "분양"
            pct = row['vectorized']/row['total']*100 if row['total'] > 0 else 0
            print(f"  - {cat_name}: {row['vectorized']}/{row['total']} ({pct:.1f}%)")
        
        print("\nDB 연결 성공")
        print("=" * 60)
        
        await conn.close()
        
    except Exception as e:
        print(f"DB 연결 실패: {e}")
        print("\n확인사항:")
        print("1. Docker 컨테이너가 실행 중인지 확인: docker ps")
        print("2. DB 설정이 올바른지 확인 (host, port, user, password)")

# DB 통계 확인
await show_db_stats()

데이터베이스 통계
총 공고 수: 473개
벡터화 완료: 473개 (100.0%)
총 청크 수: 20,352개
평균 청크/공고: 43.0개

분류별 벡터화 현황:
  - 분양: 74/74 (100.0%)
  - 임대: 399/399 (100.0%)

DB 연결 성공


## 3. 벡터 검색 함수

In [4]:
async def search_similar_chunks(query: str, top_k: int = 5, filters: dict = None):
    """
    쿼리와 유사한 문서 청크 검색
    
    Args:
        query: 검색 쿼리
        top_k: 반환할 최대 결과 수
        filters: 필터 조건 (예: {'region': '서울', 'category': 'lease'})
    
    Returns:
        검색 결과 리스트
    """
    # 쿼리 임베딩
    query_embedding = model.encode(query, normalize_embeddings=True)
    query_embedding_str = str(query_embedding.tolist())
    
    conn = await asyncpg.connect(**DB_CONFIG)
    
    try:
        # SQL 쿼리 구성
        where_clauses = []
        params = [query_embedding_str]
        
        if filters:
            if 'region' in filters:
                where_clauses.append(f"a.region LIKE ${len(params)+1}")
                params.append(f"%{filters['region']}%")
            
            if 'category' in filters:
                where_clauses.append(f"a.category = ${len(params)+1}")
                params.append(filters['category'])
            
            if 'notice_type' in filters:
                where_clauses.append(f"a.notice_type LIKE ${len(params)+1}")
                params.append(f"%{filters['notice_type']}%")
        
        where_sql = " AND " + " AND ".join(where_clauses) if where_clauses else ""
        params.append(top_k)
        
        sql = f"""
            SELECT
                dc.announcement_id,
                a.title,
                a.category,
                a.region,
                a.notice_type,
                dc.chunk_text,
                dc.metadata,
                1 - (dc.embedding <=> $1::vector) as similarity
            FROM document_chunks dc
            JOIN announcements a ON dc.announcement_id = a.id
            WHERE 1=1 {where_sql}
            ORDER BY dc.embedding <=> $1::vector
            LIMIT ${len(params)}
        """
        
        results = await conn.fetch(sql, *params)
        return results
    
    finally:
        await conn.close()


def display_results(results):
    """검색 결과를 보기 좋게 출력"""
    if not results:
        print("검색 결과가 없습니다.")
        return
    
    for idx, row in enumerate(results, 1):
        print(f"\n{'='*80}")
        print(f"[{idx}] {row['title']}")
        
        category_name = "임대" if row['category'] == 'lease' else "분양"
        print(f"분류: {category_name} | 지역: {row['region']} | 유형: {row['notice_type'] or 'N/A'}")
        print(f"유사도: {row['similarity']:.2%}")
        
        # 청크 내용 미리보기
        preview_length = 300
        chunk_preview = row['chunk_text'][:preview_length]
        if len(row['chunk_text']) > preview_length:
            chunk_preview += "..."
        print(f"\n{chunk_preview}")
        
        # 메타데이터 파싱
        metadata = row['metadata']
        if isinstance(metadata, str):
            metadata = json.loads(metadata)
        print(f"\n파일: {metadata.get('file_name', 'N/A')}")

## 4. 벡터 검색 예제

In [5]:
# 예제 1: 기본 검색
query = "서울 강남구 행복주택 신청 자격"
print(f"검색: {query}\n")

results = await search_similar_chunks(query, top_k=3)
display_results(results)

검색: 서울 강남구 행복주택 신청 자격


[1] [정정공고]구리시, 남양주시 지역 행복주택 예비입주자 모집
분류: 임대 | 지역: 경기도 | 유형: 행복주택
유사도: 67.69%

***�만65세�고령자�중�스마트폰미사용자,타인명의�휴대폰�사용자등에�한하여�인터넷,�모바일,�현장대행접수�중�하나의�방법을�**

**선택하여�접수하시기�바라며,�현장대행접수로�접수하실�경우�** **★2025년�5월�8일(목)에만�접수가능�하오니�이점�유의하여�**

**주시기�바랍니다.�** **반드시�대행접수�신청서�작성하여�방문할�것(신청서�공고�붙임파일�첨부)�**

***** � **인터넷·모바일�청약�신청자는�서류제출대상자�발표일(2025.05.22(목) 17시�이후)에�서류제출대상자�선정여부를���**

**��직접...

파일: (LH_lease_636)_★구리시,남양주시행복주택예비입주자모집(25.04.21공고).pdf

[2] 구리시, 남양주시 지역 행복주택 예비입주자 모집
분류: 임대 | 지역: 경기도 | 유형: 행복주택
유사도: 67.69%

***�만65세�고령자�중�스마트폰미사용자,타인명의�휴대폰�사용자등에�한하여�인터넷,�모바일,�현장대행접수�중�하나의�방법을�**

**선택하여�접수하시기�바라며,�현장대행접수로�접수하실�경우�** **★2025년�5월�8일(목)에만�접수가능�하오니�이점�유의하여�**

**주시기�바랍니다.�** **반드시�대행접수�신청서�작성하여�방문할�것(신청서�공고�붙임파일�첨부)�**

***** � **인터넷·모바일�청약�신청자는�서류제출대상자�발표일(2025.05.22(목) 17시�이후)에�서류제출대상자�선정여부를���**

**��직접...

파일: (LH_lease_632)_★구리시,남양주시행복주택예비입주자모집(25.04.21공고).pdf

[3] [신규모집]부천원종 A2블록 행복주택 입주자모집
분류: 임대 | 지역: 경기도 | 유형: 행복주택
유사도: 66.40%

자격요건 및 자격서류의 발급가능 여부를 충분히 확인 후

In [6]:
# 예제 2: 지역 필터 검색
query = "국민임대주택 소득 기준"
filters = {'region': '경기'}

print(f"검색: {query}")
print(f"필터: 경기 지역\n")

results = await search_similar_chunks(query, top_k=3, filters=filters)
display_results(results)

검색: 국민임대주택 소득 기준
필터: 경기 지역


[1] 수원매산 A1블록 행복주택 입주자 모집 공고
분류: 임대 | 지역: 경기도 | 유형: 행복주택
유사도: 72.89%

|Col1|거주 중 소득기준을 초과한 경우에는 임대차계약기간 종료시점 기준으로 산정된 공급대상자별<br>•<br>표준임대보증금 및 표준임대료에 아래의 할증비율로 산출된 금액이 적용됩니다<br>.<br>* 「공공주택 특별법 시행규칙 별표5 제2호 가목에 따른 철거민 등 기존거주자 중 해당 공급대상의<br>」<br>자격을 갖추지 못한 경우에는 해당 시점의 표준임대보증금 및 표준임대료에 해당 세대의 월평균<br>소득이 전년도 도시근로자 가구원수별 가구당 월평균소득 100%를 초과하는 비율에 따라 아래의<br>할증비율로 산출된 금액이 적용...

파일: (LH_lease_249)_(공고문)수원매산_A1블록_행복주택_최초모집(12.31).pdf

[2] 평택안중2 국민임대주택 예비입주자모집(2025.07.07공고분)
분류: 임대 | 지역: 경기도 | 유형: 국민임대
유사도: 72.58%

|Col1|가구원수별가구당월평균소득70%(1인가구는90% 2인가구는80%)이하이며영구<br>,<br>임대주택자산요건을충족하는자<br>자 만65세이상인사람으로서국민기초생활보장법상수급권자또는차상위계층<br>.<br>에해당하는자<br>카 기타 국토교통부장관 시 ∙도지사가 영구임대주택의 입주가 필요하다고 인정하는 자<br>.,|
|---|---|
||ㆍ「국민기초생활 보장법」 제2조제10호에 따른 차상위계층에 속한 사람(생계‧<br>의료급여 이외의 수급자 및 그 가구원 포함)**: 3점**|
||ㆍ영구임대주택에 거주하는 자(계약자에 한함)중...

파일: (LH_lease_1018)_평택지역(안중)국민임대주택표준입주자모집공고문(25.07.07공고).pdf

[3] 성남신촌 A1블록 통합공공임대주택 입주자 모집공고
분류: 임대 | 지역: 경기도 | 유형: 통합공공임대
유사도: 71.69%

- 8인을 초과하는 가구의 기준

In [7]:
# 예제 3: 청약저축 관련
query = "청약저축 납입 횟수 배점 기준"

print(f"검색: {query}\n")
results = await search_similar_chunks(query, top_k=3)
display_results(results)

검색: 청약저축 납입 횟수 배점 기준


[1] 김포시 국민임대주택 예비입주자 모집[선계약후검증]
분류: 임대 | 지역: 경기도 | 유형: 국민임대
유사도: 76.36%

① 1순위 : 주택청약종합저축에 가입하여 24회 이상 납입한 사람

② 2순위 : 주택청약종합저축에 가입하여 6회 이상 납입한 사람

③ 3순위 : 제1순위 및 제2순위에 해당되지 아니하는 자

- 청약저축가입자는 주택청약종합저축가입자로 봄

- **청약납입횟수는 통장 납입횟수가 아닌 청약순위확인서에 의함**

- 당첨되더라도 다른 분양주택 또는 임대주택을 청약하는데 청약통장을 재사용할 수 있음

2. **위** **순위 내에서 경쟁이 있을 경우 배점이 높은 순으로 선정**

3. **경쟁 발생 시 추첨으로 선정**

**■ 배점기...

파일: (LH_lease_1292)_20250905국민임대주택예비입주자모집공고문(선계약후검증).pdf

[2] 화성시 지역 국민임대주택 입주자 모집 공고(입주자격완화, 선계약후검증)
분류: 임대 | 지역: 경기도 | 유형: 국민임대
유사도: 75.82%

|② 청약저축(주택청약종합저축) 납입횟수|가 6회 이상 12회 이하 : 1점 / 나 13회 이상 24회 이하 : 2점<br>. .<br>다 25회 이상 36회 이하 : 3점 / 라 37회 이상 48회 이하 : 4점<br>. .<br>마 49회 이상 60회 이하 : 5점 / 바 61회 이상 : 6점<br>. .|
|---|---|
|③ 미성년자녀수(태아 및 배우자의 전혼자녀, 조부<br>모와 손자녀로만 구성된 세대의 손자녀 포함)<br> * 만19세미만(2006.10.17.이후 출생자녀)|가. 2자녀 :**2점** / 나. 3자녀 이...

파일: (LH_lease_1500)_화성시지역국민임대주택예비입주자모집공고[입주자격완화,선계약후검증](2025.10.16).pdf

[3] [정정공고]2025년 3월 고양시 지역 국민임대주택 예비입주자 모집공고
분류: 임대 | 지역: 경기도 | 유형: 국민임대
유사도: 7

## 5. OpenAI를 활용한 RAG 질의응답

검색된 문서를 바탕으로 GPT-4o-mini가 자연스러운 답변을 생성합니다.

In [8]:
# OpenAI 설정
from openai import OpenAI
import os
from dotenv import load_dotenv

# .env 파일에서 API 키 로드
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', '')

if not OPENAI_API_KEY:
    print("[경고] OPENAI_API_KEY가 설정되지 않았습니다.")
    print("프로젝트 루트에 .env 파일을 생성하고 다음 형식으로 API 키를 추가하세요:")
    print("OPENAI_API_KEY=sk-...")
else:
    print("OpenAI API 키가 설정되었습니다.")
    client = OpenAI(api_key=OPENAI_API_KEY)

OpenAI API 키가 설정되었습니다.


In [9]:
async def rag_qa(question: str, top_k: int = 5, filters: dict = None, model: str = "gpt-4o-mini"):
    """
    LLM 기반 RAG 질의응답
    
    Args:
        question: 사용자 질문
        top_k: 참고할 문서 청크 수
        filters: 검색 필터 (예: {'region': '서울'})
        model: 사용할 OpenAI 모델 (기본: gpt-4o-mini)
    
    Returns:
        (answer, search_results) 튜플
    """
    print(f"\n{'='*80}")
    print(f"질문: {question}")
    if filters:
        print(f"필터: {filters}")
    print(f"{'='*80}\n")
    
    # 1. 관련 문서 검색
    print("관련 공고 검색 중...\n")
    results = await search_similar_chunks(question, top_k=top_k, filters=filters)
    
    if not results:
        print("관련 정보를 찾을 수 없습니다.")
        return None, []
    
    print(f"{len(results)}개의 관련 문서를 찾았습니다.\n")
    
    # 2. 컨텍스트 구성
    context_parts = []
    for idx, r in enumerate(results, 1):
        metadata = r['metadata']
        if isinstance(metadata, str):
            metadata = json.loads(metadata)
        
        category_name = "임대" if r['category'] == 'lease' else "분양"
        
        context_parts.append(f"""
[문서 {idx}]
공고명: {r['title']}
분류: {category_name}
지역: {r['region']}
유형: {r['notice_type'] or 'N/A'}
출처: {metadata.get('file_name', 'N/A')}
관련도: {r['similarity']:.1%}

내용:
{r['chunk_text']}
        """.strip())
    
    context = "\n\n" + ("="*80 + "\n\n").join(context_parts)
    
    # 3. LLM 프롬프트 구성
    system_prompt = """당신은 LH 공사의 임대/분양 공고 전문 상담사입니다.
사용자의 질문에 대해 제공된 공고 문서를 바탕으로 정확하고 친절하게 답변해주세요.

답변 가이드:
1. 제공된 문서의 정보만을 근거로 답변하세요
2. 정보가 부족하면 "제공된 공고에서는 해당 정보를 찾을 수 없습니다"라고 명시하세요
3. 구체적인 수치, 날짜, 조건 등을 정확히 인용하세요
4. 여러 공고를 비교할 때는 각 공고명을 명시하세요
5. 전문 용어는 알기 쉽게 설명해주세요
6. 가독성 좋게 마크다운 형식으로 작성하세요 (제목, 목록, 표 등 활용)"""

    user_prompt = f"""다음은 사용자의 질문과 관련된 LH 공고 문서입니다.

{context}

사용자 질문: {question}

위 문서를 바탕으로 질문에 답변해주세요."""

    # 4. OpenAI API 호출
    try:
        print("AI 답변 생성 중...\n")
        
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.2,
            max_tokens=2000
        )
        
        answer = response.choices[0].message.content
        
        # 5. 결과 출력
        print("="*80)
        print("답변")
        print("="*80)
        print(answer)
        print("\n" + "="*80)
        print("참고 공고")
        print("="*80)
        
        for idx, r in enumerate(results, 1):
            category_name = "임대" if r['category'] == 'lease' else "분양"
            print(f"{idx}. {r['title']}")
            print(f"   - {category_name} | {r['region']} | 관련도: {r['similarity']:.1%}")
        
        print(f"\n토큰 사용량: {response.usage.total_tokens:,} (입력: {response.usage.prompt_tokens:,}, 출력: {response.usage.completion_tokens:,})")
        print(f"모델: {model}")
        print("="*80)
        
        return answer, results
        
    except Exception as e:
        print(f"오류 발생: {str(e)}")
        if "api_key" in str(e).lower():
            print("\nOPENAI_API_KEY를 확인해주세요.")
        return None, results

### RAG 질의응답 예제

In [10]:
# 예제 1: 입주 자격 조건
answer, results = await rag_qa(
    "남양주시 국민임대주택의 입주 자격 조건은 무엇인가요?",
    top_k=3
)


질문: 남양주시 국민임대주택의 입주 자격 조건은 무엇인가요?

관련 공고 검색 중...

3개의 관련 문서를 찾았습니다.

AI 답변 생성 중...

답변
제공된 공고에서는 남양주시 국민임대주택의 입주 자격 조건에 대한 정보가 포함되어 있지 않습니다. 

따라서, 남양주시 국민임대주택의 입주 자격 조건에 대해서는 "제공된 공고에서는 해당 정보를 찾을 수 없습니다"라고 말씀드립니다. 추가적인 정보가 필요하시면 LH 공사의 공식 웹사이트나 관련 부서에 문의하시기 바랍니다.

참고 공고
1. 성남금토 A-2블록 영구임대주택 입주자 모집
   - 임대 | 경기도 | 관련도: 66.7%
2. [경기남부] 25년 3차 청년 매입임대주택 예비입주자 모집공고
   - 임대 | 경기도 | 관련도: 66.2%
3. [정정공고][경기남부] 25년 3차 청년 매입임대주택 예비입주자 모집공고
   - 임대 | 경기도 | 관련도: 65.4%

토큰 사용량: 2,443 (입력: 2,350, 출력: 93)
모델: gpt-4o-mini


In [11]:
# 예제 2: 지역별 공고 비교
answer, results = await rag_qa(
    "수원시 지역 행복주택은 어떤 것들이 있고, 각각의 특징은 무엇인가요?",
    top_k=5,
    filters={'region': '수원'}
)


질문: 수원시 지역 행복주택은 어떤 것들이 있고, 각각의 특징은 무엇인가요?
필터: {'region': '수원'}

관련 공고 검색 중...

관련 정보를 찾을 수 없습니다.


In [12]:
# 예제 3: 청약저축 배점
answer, results = await rag_qa(
    "청약저축 납입 횟수에 따른 배점 기준을 자세히 설명해주세요.",
    top_k=3
)


질문: 청약저축 납입 횟수에 따른 배점 기준을 자세히 설명해주세요.

관련 공고 검색 중...

3개의 관련 문서를 찾았습니다.

AI 답변 생성 중...

답변
청약저축 납입 횟수에 따른 배점 기준은 다음과 같습니다. 두 개의 공고에서 각각의 배점 기준을 확인할 수 있습니다.

### 1. 김포시 국민임대주택 예비입주자 모집 공고
- **청약저축 가입자**는 주택청약종합저축 가입자로 간주됩니다.
- 청약 순위 확인서에 의한 청약 납입 횟수에 따라 순위가 결정됩니다.
- 순위는 다음과 같습니다:
  - **1순위**: 주택청약종합저축에 가입하여 24회 이상 납입한 사람
  - **2순위**: 주택청약종합저축에 가입하여 6회 이상 납입한 사람
  - **3순위**: 제1순위 및 제2순위에 해당되지 않는 자

### 2. 화성시 지역 국민임대주택 입주자 모집 공고
- 청약저축(주택청약종합저축) 납입 횟수에 따른 배점 기준은 다음과 같습니다:
  | 납입 횟수 구간 | 배점 |
  |----------------|------|
  | 6회 이상 12회 이하 | 1점 |
  | 13회 이상 24회 이하 | 2점 |
  | 25회 이상 36회 이하 | 3점 |
  | 37회 이상 48회 이하 | 4점 |
  | 49회 이상 60회 이하 | 5점 |
  | 61회 이상 | 6점 |

### 요약
- **김포시**에서는 청약 납입 횟수에 따라 1순위, 2순위, 3순위로 나뉘며, 1순위가 가장 우선합니다.
- **화성시**에서는 청약 납입 횟수에 따라 점수가 부여되며, 더 많은 납입 횟수에 따라 높은 점수를 받을 수 있습니다.

이 정보를 바탕으로 청약 신청 시 유리한 조건을 고려하시기 바랍니다. 추가적인 질문이 있으시면 언제든지 문의해 주세요!

참고 공고
1. 김포시 국민임대주택 예비입주자 모집[선계약후검증]
   - 임대 | 경기도 | 관련도: 77.8%
2. 화성시 지역 국민임대주택 입주자 모집 공고(입주자격완화, 선계약후검증)
   - 임대 | 경기도 | 관련도:

## 6. 커스텀 질문 실행

아래 셀에서 자유롭게 질문을 입력하고 실행해보세요.

In [13]:
# 여기에 질문을 입력하세요
my_question = "영구임대주택과 국민임대주택의 차이점은 무엇인가요?"

# 필요시 필터 설정 (예: {'region': '서울', 'category': 'lease'})
my_filters = None

# 검색할 문서 수 (기본: 5)
my_top_k = 5

# 실행
answer, results = await rag_qa(
    question=my_question,
    top_k=my_top_k,
    filters=my_filters
)


질문: 영구임대주택과 국민임대주택의 차이점은 무엇인가요?

관련 공고 검색 중...

5개의 관련 문서를 찾았습니다.

AI 답변 생성 중...

답변
## 영구임대주택과 국민임대주택의 차이점

영구임대주택과 국민임대주택은 모두 임대주택의 일종이지만, 몇 가지 중요한 차이점이 있습니다.

### 1. **임대 기간**
- **영구임대주택**: 이름에서 알 수 있듯이, 영구적으로 임대가 가능한 주택입니다. 즉, 입주자는 장기적으로 거주할 수 있습니다.
- **국민임대주택**: 일반적으로 일정 기간 동안 임대가 가능하며, 보통 10년에서 30년의 임대 기간이 설정됩니다. 임대 기간이 종료되면 재계약이나 다른 주택으로의 이주가 필요할 수 있습니다.

### 2. **대상**
- **영구임대주택**: 주로 저소득층, 장애인, 노인 등 사회적 약자를 대상으로 하며, 특정 자격 요건을 충족해야 합니다.
- **국민임대주택**: 국민 전체를 대상으로 하며, 소득 기준에 따라 차상위계층 및 저소득층이 우선적으로 입주할 수 있습니다.

### 3. **임대료**
- **영구임대주택**: 상대적으로 저렴한 임대료가 책정되며, 소득 수준에 따라 차등 적용될 수 있습니다.
- **국민임대주택**: 임대료는 시장 가격보다 낮지만, 영구임대주택보다는 상대적으로 높을 수 있습니다.

### 4. **주택 유형**
- **영구임대주택**: 영구임대주택은 주로 영구적으로 거주할 수 있는 주택으로 구성되어 있습니다.
- **국민임대주택**: 국민임대주택은 일반적으로 다양한 유형의 주택이 포함되어 있으며, 특정 지역의 주택 수요에 따라 다르게 구성될 수 있습니다.

### 5. **입주자 모집 방식**
- **영구임대주택**: 특정 자격 요건을 충족하는 경우에 한해 모집됩니다.
- **국민임대주택**: 보다 넓은 범위의 국민을 대상으로 하며, 주택청약종합저축 가입자 등 다양한 조건을 고려하여 모집합니다.

이와 같은 차이점으로 인해, 영구임대주택과 국민임대주택은 각각의 필요에 따라 선택할 수

## 7. 고급 활용 예제

In [14]:
# 대화형 질의응답 (여러 번 질문 가능)
conversation_history = []

async def chat(question: str, top_k: int = 5):
    """대화 이력을 유지하며 질의응답"""
    answer, results = await rag_qa(question, top_k=top_k)
    
    if answer:
        conversation_history.append({
            'question': question,
            'answer': answer,
            'sources': [r['title'] for r in results]
        })
    
    return answer, results

# 사용 예시
# await chat("서울 강남구 행복주택 알려줘")
# await chat("거기 소득 기준은?")  # 이전 대화 맥락 유지

In [15]:
# 대화 이력 확인
print("대화 이력\n")
for idx, conv in enumerate(conversation_history, 1):
    print(f"[{idx}] Q: {conv['question']}")
    print(f"    참고 공고: {', '.join(conv['sources'][:2])}...\n")

대화 이력



## 8. 유틸리티 함수

In [16]:
async def get_announcement_list(filters: dict = None, limit: int = 20):
    """공고 목록 조회"""
    conn = await asyncpg.connect(**DB_CONFIG)
    
    try:
        where_clauses = ["is_vectorized = TRUE"]
        params = []
        
        if filters:
            if 'region' in filters:
                where_clauses.append(f"region LIKE ${len(params)+1}")
                params.append(f"%{filters['region']}%")
            
            if 'category' in filters:
                where_clauses.append(f"category = ${len(params)+1}")
                params.append(filters['category'])
        
        where_sql = " AND ".join(where_clauses)
        params.append(limit)
        
        sql = f"""
            SELECT id, title, category, region, notice_type, created_at
            FROM announcements
            WHERE {where_sql}
            ORDER BY created_at DESC
            LIMIT ${len(params)}
        """
        
        results = await conn.fetch(sql, *params)
        
        # DataFrame으로 변환
        df = pd.DataFrame([dict(r) for r in results])
        if not df.empty:
            df['category'] = df['category'].map({'lease': '임대', 'sale': '분양'})
        
        return df
        
    finally:
        await conn.close()

# 사용 예시
announcements = await get_announcement_list(filters={'region': '서울'}, limit=10)
display(announcements)

,id,title,category,region,notice_type,created_at
0,LH_lease_1598,[정정공고]도봉주공1단지 국민임대주택 예비입주자 모집 공고,임대,서울특별시,국민임대,2025-11-19 18:54:21.304618
1,LH_lease_1593,2025년 서울특별시 영구임대주택 예비입주자 모집,임대,서울특별시,영구임대,2025-11-19 18:54:21.303767
2,LH_lease_1538,[정정공고]서울 청년 특화형 매입임대주택 [강북구 위너스빌] 예비입주자 모집공고문[...,임대,서울특별시,매입임대,2025-11-19 18:54:21.280662
3,LH_lease_1518,[서울지역본부] 25년 2차 비분양전환형 든든전세 입주자 모집공고,임대,서울특별시,매입임대,2025-11-19 18:54:21.277674
4,LH_lease_1508,서울 청년 특화형 매입임대주택 [강북구 위너스빌] 예비입주자 모집공고문[25.10.17],임대,서울특별시,매입임대,2025-11-19 18:54:21.274639
5,LH_lease_1481,2025 다자녀 전세임대 입주자 수시모집 공고,임대,서울특별시 외,전세임대,2025-11-19 18:54:21.268608
6,LH_lease_1480,2025년 신혼·신생아 전세임대Ⅱ 입주자 수시모집 공고,임대,서울특별시 외,전세임대,2025-11-19 18:54:21.267926
7,LH_lease_1479,2025년 청년 전세임대 1순위 입주자 수시모집,임대,서울특별시 외,전세임대,2025-11-19 18:54:21.267133
8,LH_lease_1478,2025년 신혼·신생아 전세임대 I 입주자 수시모집 공고,임대,서울특별시 외,전세임대,2025-11-19 18:54:21.266237
9,LH_lease_1431,[정정공고][서울지역본부] 25년 3차 청년 매입임대주택 예비입주자 모집공고,임대,서울특별시,매입임대,2025-11-19 18:54:21.256263


---

## 참고사항

### 검색 팁

1. **구체적인 질문이 더 좋은 결과를 얻습니다**
   - 나쁜 예: "집 알려줘"
   - 좋은 예: "서울 강남구 행복주택 신청 자격 조건 알려줘"

2. **필터 활용으로 검색 범위를 좁힐 수 있습니다**
   ```python
   filters = {
       'region': '서울',          # 지역 필터
       'category': 'lease',       # 'lease'(임대) 또는 'sale'(분양)
       'notice_type': '행복주택'  # 공고 유형
   }
   ```

3. **top_k 조정으로 검색 결과 수를 변경할 수 있습니다**
   - 기본값: 5
   - 복잡한 질문: 10~15 추천
   - 단순한 질문: 3~5 추천

### 비용 안내

- GPT-4o-mini 사용 시 쿼리당 약 0.001~0.01원 수준
- 토큰 사용량은 각 답변 하단에 표시됩니다

### 문제 해결

**1. DB 연결 오류**
```bash
docker ps                  # 컨테이너 확인
docker-compose up -d       # 재시작
```

**2. OpenAI API 오류**
- .env 파일의 API 키를 확인하세요

**3. 검색 결과 없음**
- 필터 조건을 완화하거나 질문을 수정해보세요